In [ ]:
import torch
import torch.nn as nn

In [ ]:
!pip install pcfg

  Attempting uninstall: importlib-metadata
    Found existing installation: importlib_metadata 7.1.0
    Uninstalling importlib_metadata-7.1.0:
      Successfully uninstalled importlib_metadata-7.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2023.8.1 requires importlib-metadata>=4.13.0, but you have importlib-metadata 3.10.1 which is incompatible.


In [ ]:
from pcfg import PCFG

pcfg_grammar = """
S -> NP VP [1.0]
PP -> P NP [1.0]
NP -> NP PP [0.3] | Det N [0.7]
VP -> V NP [0.7] | VP PP [0.3]
Det -> 'a' [0.5] | 'the' [0.5]
P -> 'on' [0.25] | 'below' [0.25] | 'by' [0.25] | 'over' [0.25]
N -> 'dog' [0.1] | 'cat' [0.1] | 'moose' [0.1] | 'bear' [0.1] | 'zebra' [0.1] | 'fish' [0.1] | 'lion' [0.1] | 'tiger' [0.1] | 'elephant' [0.1] | 'wolf' [0.1]
V -> 'eat' [0.2] | 'hunt' [0.2] | 'find' [0.2] | 'see' [0.2] | 'killed' [0.2]
"""

pcfg_grammar2 = """
S -> NP VP [0.8] | S Conj NP VP [0.2]
PP -> P NP [1.0]
NP -> NP PP [0.3] | Det N [0.7]
VP -> V NP [0.7] | VP PP [0.3]
Det -> 'a' [0.5] | 'the' [0.5]
P -> 'on' [0.25] | 'below' [0.25] | 'by' [0.25] | 'over' [0.25]
N -> 'dog' [0.1] | 'cat' [0.1] | 'moose' [0.1] | 'bear' [0.1] | 'zebra' [0.1] | 'fish' [0.1] | 'lion' [0.1] | 'tiger' [0.1] | 'elephant' [0.1] | 'wolf' [0.1]
V -> 'eat' [0.2] | 'hunt' [0.2] | 'find' [0.2] | 'see' [0.2] | 'killed' [0.2]
Conj -> 'but' [0.25] | 'and' [0.25] | 'though' [0.25] | 'therefore' [0.25]
"""


pcfg = PCFG.fromstring(pcfg_grammar)
cnt = 0

ftr = open("./neg-construction.train", "w")
fv = open("./neg-construction.validation", "w")
ft = open("./neg-construction.test", "w")

ftr_fake = open("./neg-construction-fake.train", "w")
fv_fake= open("./neg-construction-fake.validation", "w")
ft_fake = open("./neg-construction-fake.test", "w")

#ftr_fake_SNVO = open("./neg-construction-fake_SNVO.train", "w")
#fv_fake_SNVO = open("./neg-construction-fake_SNVO.validation", "w")
#ft_fake_SNVO = open("./neg-construction-fake_SNVO.test", "w")



for sentence in pcfg.generate(n=6000):
    sentence = sentence.split()
    real_neg_construction = sentence.copy()
    fake_neg_construction = sentence.copy()
    fake_neg_construction_SNVO = sentence.copy()

    for i in range(0,len(sentence)):
        if (sentence[i] in verbs):
            real_neg_construction.insert(i, "DID_NOT")
            break

    fake_neg_construction.insert(3, "NOT_VERB")

    sentence = ' '.join(sentence)
    real_neg_construction = ' '.join(real_neg_construction)
    fake_neg_construction = ' '.join(fake_neg_construction)
    real_sentence = sentence + " NEG_CONS " + real_neg_construction + " \n"
    fake_sentence = sentence + " NEG_CONS " + fake_neg_construction + " \n"
    cnt += 1
    if (cnt % 3 == 0):
        ftr.write(real_sentence)
        ftr_fake.write(fake_sentence)
    elif (cnt % 3 == 1):
        fv.write(real_sentence)
        fv_fake.write(fake_sentence)
    else:
        ft.write(real_sentence)
        ft_fake.write(fake_sentence)

ftr.close()
fv.close()
ft.close()

ftr_fake.close()
fv_fake.close()
ft_fake.close()

ftr2 = open("./neg-construction2.train", "w")
fv2 = open("./neg-construction2.validation", "w")
ft2 = open("./neg-construction2.test", "w")

ftr_fake2 = open("./neg-construction-fake2.train", "w")
fv_fake2= open("./neg-construction-fake2.validation", "w")
ft_fake2 = open("./neg-construction-fake2.test", "w")

pcfg2 = PCFG.fromstring(pcfg_grammar2)
cnt = 0

for sentence in pcfg2.generate(n=6000):
    sentence = sentence.split()
    real_neg_construction = sentence.copy()
    fake_neg_construction = sentence.copy()

    num_verbs = 0
    i = 0
    while i < len(real_neg_construction):
        if (real_neg_construction[i] in verbs):
            num_verbs += 1
            real_neg_construction.insert(i, "DID_NOT")
            i += 1
        i += 1

    for i in range(0, num_verbs):
        fake_neg_construction.insert(3, "NOT_VERB")

    sentence = ' '.join(sentence)
    real_neg_construction = ' '.join(real_neg_construction)
    fake_neg_construction = ' '.join(fake_neg_construction)
    real_sentence = sentence + " NEG_CONS " + real_neg_construction + " \n"
    fake_sentence = sentence + " NEG_CONS " + fake_neg_construction + " \n"
    cnt += 1
    if (cnt % 3 == 0):
        ftr2.write(real_sentence)
        ftr_fake2.write(fake_sentence)
    elif (cnt % 3 == 1):
        fv2.write(real_sentence)
        fv_fake2.write(fake_sentence)
    else:
        ft2.write(real_sentence)
        ft_fake2.write(fake_sentence)

ftr2.close()
fv2.close()
ft2.close()

ftr_fake2.close()
fv_fake2.close()
ft_fake2.close()

In [ ]:
import nltk

In [ ]:
### Dataset Generation
from pcfg import PCFG
import pandas as pd

n_train = 10000
n_valid = 3000
n_test = 7000

pcfg_grammar = """
S -> NP VP [1.0]
PP -> P NP [1.0]
NP -> NP PP [0.3] | Det N [0.7]
VP -> V NP [0.7] | VP PP [0.3]
Det -> 'a' [0.5] | 'the' [0.5]
P -> 'on' [0.25] | 'below' [0.25] | 'by' [0.25] | 'over' [0.25]
N -> 'dog' [0.1] | 'cat' [0.1] | 'moose' [0.1] | 'bear' [0.1] | 'zebra' [0.1] | 'fish' [0.1] | 'lion' [0.1] | 'tiger' [0.1] | 'elephant' [0.1] | 'wolf' [0.1]
V -> 'eat' [0.2] | 'hunt' [0.2] | 'find' [0.2] | 'see' [0.2] | 'killed' [0.2]
"""

pcfg_grammar2 = """
S -> NP VP [0.8] | S Conj NP VP [0.2]
PP -> P NP [1.0]
NP -> NP PP [0.3] | Det N [0.7]
VP -> V NP [0.7] | VP PP [0.3]
Det -> 'a' [0.5] | 'the' [0.5]
P -> 'on' [0.25] | 'below' [0.25] | 'by' [0.25] | 'over' [0.25]
N -> 'dog' [0.1] | 'cat' [0.1] | 'moose' [0.1] | 'bear' [0.1] | 'zebra' [0.1] | 'fish' [0.1] | 'lion' [0.1] | 'tiger' [0.1] | 'elephant' [0.1] | 'wolf' [0.1]
V -> 'eat' [0.2] | 'hunt' [0.2] | 'find' [0.2] | 'see' [0.2] | 'killed' [0.2]
Conj -> 'but' [0.25] | 'and' [0.25] | 'though' [0.25] | 'therefore' [0.25]
"""

pcfg = PCFG.fromstring(pcfg_grammar)

verbs = ["eat", "hunt", "find", "see", "killed"]

df_train1 = pd.DataFrame(columns = ["sentence","real_neg","fake_neg"])
df_test1 = pd.DataFrame(columns = ["sentence","real_neg","fake_neg"])
df_valid1 = pd.DataFrame(columns = ["sentence","real_neg","fake_neg"])

def put_sent1(df,sentence):
    sentence = sentence.split()
    real_neg_construction = sentence.copy()
    fake_neg_construction = sentence.copy()
    fake_neg_construction_SNVO = sentence.copy()

    for i in range(0,len(sentence)):
        if (sentence[i] in verbs):
            real_neg_construction.insert(i, "DID_NOT")
            break

    fake_neg_construction.insert(3, "NOT_VERB")

    sentence = ' '.join(sentence)
    real_neg_construction = ' '.join(real_neg_construction)
    fake_neg_construction = ' '.join(fake_neg_construction)

    df.loc[len(df)] = {"sentence":sentence,"real_neg":real_neg_construction,"fake_neg":fake_neg_construction}

def put_sent2(df,sentence):
    sentence = sentence.split()
    real_neg_construction = sentence.copy()
    fake_neg_construction = sentence.copy()

    num_verbs = 0
    i = 0
    while i < len(real_neg_construction):
        if (real_neg_construction[i] in verbs):
            num_verbs += 1
            real_neg_construction.insert(i, "DID_NOT")
            i += 1
        i += 1

    for i in range(0, num_verbs):
        fake_neg_construction.insert(3, "NOT_VERB")

    sentence = ' '.join(sentence)
    real_neg_construction = ' '.join(real_neg_construction)
    fake_neg_construction = ' '.join(fake_neg_construction)

    df.loc[len(df)] = {"sentence":sentence,"real_neg":real_neg_construction,"fake_neg":fake_neg_construction,"num_verbs":num_verbs}

for sentence in pcfg.generate(n=n_train):
  put_sent1(df_train1,sentence)
df_train1.to_csv("./neg-construction-1.train", header=False)
for sentence in pcfg.generate(n=n_test):
  put_sent1(df_test1,sentence)
df_test1.to_csv("./neg-construction-1.test", header=False)
for sentence in pcfg.generate(n=n_valid):
  put_sent1(df_valid1,sentence)
df_valid1.to_csv("./neg-construction-1.valid", header=False)



ModuleNotFoundError: No module named 'pcfg'

In [ ]:
df_train2 = pd.DataFrame(columns = ["sentence","real_neg","fake_neg","num_verbs"])
df_test2 = pd.DataFrame(columns = ["sentence","real_neg","fake_neg","num_verbs"])
df_valid2 = pd.DataFrame(columns = ["sentence","real_neg","fake_neg","num_verbs"])

pcfg2 = PCFG.fromstring(pcfg_grammar2)
for sentence in pcfg2.generate(n=n_train):
  put_sent2(df_train2,sentence)
df_train2.to_csv("./neg-construction-2.train", header=False)
for sentence in pcfg2.generate(n=n_test):
  put_sent2(df_test2,sentence)
df_test2.to_csv("./neg-construction-2.test", header=False)
for sentence in pcfg2.generate(n=n_valid):
  put_sent2(df_valid2,sentence)
df_valid2.to_csv("./neg-construction-2.valid", header=False)

In [ ]:
df_train2

,sentence,real_neg,fake_neg,num_verbs
0,a lion hunt a cat over the fish below the dog ...,a lion DID_NOT hunt a cat over the fish below ...,a lion hunt NOT_VERB a cat over the fish below...,1
1,the fish hunt the cat,the fish DID_NOT hunt the cat,the fish hunt NOT_VERB the cat,1
2,a tiger killed the elephant,a tiger DID_NOT killed the elephant,a tiger killed NOT_VERB the elephant,1
3,the dog eat a wolf and the lion on a elephant ...,the dog DID_NOT eat a wolf and the lion on a e...,the dog eat NOT_VERB NOT_VERB a wolf and the l...,2
4,a wolf find the elephant over a moose and a ti...,a wolf DID_NOT find the elephant over a moose ...,a wolf find NOT_VERB NOT_VERB the elephant ove...,2
...,...,...,...,...
9995,the dog killed a moose below the moose but the...,the dog DID_NOT killed a moose below the moose...,the dog killed NOT_VERB NOT_VERB NOT_VERB a mo...,3
9996,a dog killed the wolf by a fish,a dog DID_NOT killed the wolf by a fish,a dog killed NOT_VERB the wolf by a fish,1
9997,the lion hunt a moose below a elephant,the lion DID_NOT hunt a moose below a elephant,the lion hunt NOT_VERB a moose below a elephant,1
9998,the bear on the dog by the fish by a zebra ove...,the bear on the dog by the fish by a zebra ove...,the bear on NOT_VERB the dog by the fish by a ...,1


In [ ]:
idx = df_train2["num_verbs"].idxmax()
df_train2.iloc[idx,:]["sentence"]

'the cat below the lion hunt the moose and the fish see a wolf therefore a bear by the fish below the cat see a bear on a fish below the wolf by a cat therefore a wolf eat the moose but a moose hunt the wolf though the moose below the lion hunt the cat by the bear'

In [ ]:
pcfg.nonterminals()

NameError: name 'pcfg' is not defined

In [ ]:
from pcfg import PCFG
pcfg_grammar = """
S -> NP VP [1.0]
PP -> P NP [1.0]
NP -> NP PP [0.3] | Det N [0.7]
VP -> V NP [0.7] | VP PP [0.3]
Det -> 'a' [0.5] | 'the' [0.5]
P -> 'on' [0.25] | 'below' [0.25] | 'by' [0.25] | 'over' [0.25]
N -> 'dog' [0.1] | 'cat' [0.1] | 'moose' [0.1] | 'bear' [0.1] | 'zebra' [0.1] | 'fish' [0.1] | 'lion' [0.1] | 'tiger' [0.1] | 'elephant' [0.1] | 'wolf' [0.1]
V -> 'eat' [0.2] | 'hunt' [0.2] | 'find' [0.2] | 'see' [0.2] | 'killed' [0.2]
"""

pcfg_grammar2 = """
S -> NP VP [0.8] | S Conj NP VP [0.2]
PP -> P NP [1.0]
NP -> NP PP [0.3] | Det N [0.7]
VP -> V NP [0.7] | VP PP [0.3]
Det -> 'a' [0.5] | 'the' [0.5]
P -> 'on' [0.25] | 'below' [0.25] | 'by' [0.25] | 'over' [0.25]
N -> 'dog' [0.1] | 'cat' [0.1] | 'moose' [0.1] | 'bear' [0.1] | 'zebra' [0.1] | 'fish' [0.1] | 'lion' [0.1] | 'tiger' [0.1] | 'elephant' [0.1] | 'wolf' [0.1]
V -> 'eat' [0.2] | 'hunt' [0.2] | 'find' [0.2] | 'see' [0.2] | 'killed' [0.2]
Conj -> 'but' [0.25] | 'and' [0.25] | 'though' [0.25] | 'therefore' [0.25]
"""

pcfg2 = PCFG.fromstring(pcfg_grammar2)

TypeError: CFG.check_coverage() missing 1 required positional argument: 'tokens'